In [ ]:
import cv2
import requests
import numpy as np

url = 'http://192.168.0.179/'

if __name__ == '__main__':
    cap = cv2.VideoCapture(url + "stream")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("not ret")
            break

        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()